In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

path_root = '../..'
sys.path.append(f'{path_root}/')
from src.util.functions import Functions as F

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = f'{path_root}/data'
path_csv = f'{path_data}/csv'

csv_out_name = f'{path_csv}/covid19-{level}.csv'
pop_regions_name = f'{path_csv}/popolazione-regioni.csv'
icu_regions_name = f'{path_csv}/terapie-intensive.csv'
regions_name = f'{path_csv}/regioni.csv'

In [2]:
url_cum = F.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,note,ingressi_terapia_intensiva,note_test,note_casi,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8332,2021-03-26T17:00:00,ITA,19,Sicilia,38.115697,13.362357,799,121,920,15483,...,NaN,8.0,NaN,NaN,169808.0,0.0,1968294.0,1101564.0,ITG,ITG1
8333,2021-03-26T17:00:00,ITA,9,Toscana,43.769231,11.255889,1473,257,1730,25496,...,NaN,14.0,NaN,NaN,187035.0,1765.0,2841708.0,470506.0,ITI,ITI1
8334,2021-03-26T17:00:00,ITA,10,Umbria,43.106758,12.388247,376,57,433,4817,...,"Si fa presente che 5 dei ricoveri NON UTI, non...",2.0,NaN,NaN,50127.0,0.0,773860.0,183908.0,ITI,ITI2
8335,2021-03-26T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,24,7,31,659,...,NaN,2.0,NaN,NaN,8742.0,131.0,79901.0,11487.0,ITC,ITC2


In [4]:
list(df_raw_r.columns)

['data',
 'stato',
 'codice_regione',
 'denominazione_regione',
 'lat',
 'long',
 'ricoverati_con_sintomi',
 'terapia_intensiva',
 'totale_ospedalizzati',
 'isolamento_domiciliare',
 'totale_positivi',
 'variazione_totale_positivi',
 'nuovi_positivi',
 'dimessi_guariti',
 'deceduti',
 'casi_da_sospetto_diagnostico',
 'casi_da_screening',
 'totale_casi',
 'tamponi',
 'casi_testati',
 'note',
 'ingressi_terapia_intensiva',
 'note_test',
 'note_casi',
 'totale_positivi_test_molecolare',
 'totale_positivi_test_antigenico_rapido',
 'tamponi_test_molecolare',
 'tamponi_test_antigenico_rapido',
 'codice_nuts_1',
 'codice_nuts_2']

In [5]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'casi_da_sospetto_diagnostico', 'casi_da_screening', 'note', 'note_test', 'note_casi']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,...,0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,...,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,...,0,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,...,18,148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8332,2021-03-26T17:00:00,19,Sicilia,799,121,920,15483,16403,409,892,...,169808,3069858,1262606.0,8.0,169808.0,0.0,1968294.0,1101564.0,ITG,ITG1
8333,2021-03-26T17:00:00,9,Toscana,1473,257,1730,25496,27226,362,1425,...,188800,3312214,1725391.0,14.0,187035.0,1765.0,2841708.0,470506.0,ITI,ITI1
8334,2021-03-26T17:00:00,10,Umbria,376,57,433,4817,5250,83,320,...,50127,957768,334692.0,2.0,50127.0,0.0,773860.0,183908.0,ITI,ITI2
8335,2021-03-26T17:00:00,2,Valle d'Aosta,24,7,31,659,690,41,57,...,8873,91388,49847.0,2.0,8742.0,131.0,79901.0,11487.0,ITC,ITC2


In [6]:
# Compute most recent couple of dates
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = F.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2021-03-26', '2021-03-25')

In [7]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-26T17:00:00,1,Piemonte,3759,358,4117,31018,35135,-157,2117,...,299438,3197847,1474130.0,19.0,286434.0,13004.0,2337314.0,860533.0,ITC,ITC1
1,2021-03-26T17:00:00,2,Valle d'Aosta,24,7,31,659,690,41,57,...,8873,91388,49847.0,2.0,8742.0,131.0,79901.0,11487.0,ITC,ITC2
2,2021-03-26T17:00:00,3,Lombardia,7111,848,7959,96696,104655,2577,5077,...,718073,7942837,3463092.0,86.0,688596.0,29477.0,7140477.0,802360.0,ITC,ITC4
3,2021-03-26T17:00:00,5,Veneto,1548,257,1805,37551,39356,173,2095,...,375500,6019629,1533645.0,30.0,366931.0,8569.0,4466174.0,1553455.0,ITH,ITH3
4,2021-03-26T17:00:00,6,Friuli Venezia Giulia,650,78,728,15292,16020,52,768,...,94980,1583938,566752.0,5.0,83507.0,11473.0,1410324.0,173614.0,ITH,ITH4
5,2021-03-26T17:00:00,7,Liguria,605,67,672,5654,6326,171,487,...,87194,1204780,505596.0,8.0,87194.0,0.0,1034593.0,170187.0,ITC,ITC3
6,2021-03-26T17:00:00,8,Emilia-Romagna,3515,399,3914,67945,71859,24,2391,...,326776,4721424,1660353.0,26.0,326563.0,213.0,3919926.0,801498.0,ITH,ITH5
7,2021-03-26T17:00:00,9,Toscana,1473,257,1730,25496,27226,362,1425,...,188800,3312214,1725391.0,14.0,187035.0,1765.0,2841708.0,470506.0,ITI,ITI1
8,2021-03-26T17:00:00,10,Umbria,376,57,433,4817,5250,83,320,...,50127,957768,334692.0,2.0,50127.0,0.0,773860.0,183908.0,ITI,ITI2
9,2021-03-26T17:00:00,11,Marche,807,152,959,9153,10112,-64,674,...,85770,982389,596994.0,6.0,85770.0,0.0,887712.0,94677.0,ITI,ITI3


In [8]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,totale_casi,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2
0,2021-03-25T17:00:00,1,Piemonte,3677,357,4034,31258,35292,344,2582,...,297321,3165732,1466122.0,28.0,284644.0,12677.0,2323334.0,842398.0,ITC,ITC1
1,2021-03-25T17:00:00,2,Valle d'Aosta,28,5,33,616,649,22,61,...,8816,90571,49532.0,2.0,8711.0,105.0,79698.0,10873.0,ITC,ITC2
2,2021-03-25T17:00:00,3,Lombardia,7132,845,7977,94101,102078,1906,5046,...,712996,7882033,3449180.0,26.0,684356.0,28640.0,7100338.0,781695.0,ITC,ITC4
3,2021-03-25T17:00:00,5,Veneto,1556,256,1812,37371,39183,55,1861,...,373405,5976984,1526831.0,22.0,364863.0,8542.0,4443885.0,1533099.0,ITH,ITH3
4,2021-03-25T17:00:00,6,Friuli Venezia Giulia,645,78,723,15245,15968,-772,842,...,94212,1573489,563322.0,8.0,82998.0,11214.0,1403101.0,170388.0,ITH,ITH4
5,2021-03-25T17:00:00,7,Liguria,589,65,654,5501,6155,362,494,...,86707,1197494,502866.0,5.0,86707.0,0.0,1030081.0,167413.0,ITC,ITC3
6,2021-03-25T17:00:00,8,Emilia-Romagna,3557,402,3959,67876,71835,-21,2070,...,324394,4689356,1655616.0,30.0,324182.0,212.0,3900448.0,788908.0,ITH,ITH5
7,2021-03-25T17:00:00,9,Toscana,1470,257,1727,25137,26864,353,1620,...,187375,3286164,1715126.0,17.0,185665.0,1710.0,2826615.0,459549.0,ITI,ITI1
8,2021-03-25T17:00:00,10,Umbria,375,63,438,4729,5167,-77,113,...,49807,950265,333491.0,4.0,49807.0,0.0,769516.0,180749.0,ITI,ITI2
9,2021-03-25T17:00:00,11,Marche,804,157,961,9215,10176,80,621,...,85096,976964,594009.0,9.0,85096.0,0.0,881523.0,95441.0,ITI,ITI3


In [9]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = df_raw_r_1
# df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
# df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

In [10]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = df_raw_r_0
# df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
# df_r_0.sort_values(by='codice_regione').reset_index()

In [11]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi,casi_testati,ingressi_terapia_intensiva,totale_positivi_test_molecolare,totale_positivi_test_antigenico_rapido,tamponi_test_molecolare,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione
0,2021-03-26T17:00:00,1,Piemonte,3759,358,4117,31018,35135,-157,2117,...,3197847,1474130.0,19.0,286434.0,13004.0,2337314.0,860533.0,ITC,ITC1,4.341375e+06
1,2021-03-26T17:00:00,2,Valle d'Aosta,24,7,31,659,690,41,57,...,91388,49847.0,2.0,8742.0,131.0,79901.0,11487.0,ITC,ITC2,1.255010e+05
2,2021-03-26T17:00:00,3,Lombardia,7111,848,7959,96696,104655,2577,5077,...,7942837,3463092.0,86.0,688596.0,29477.0,7140477.0,802360.0,ITC,ITC4,1.010397e+07
3,2021-03-26T17:00:00,5,Veneto,1548,257,1805,37551,39356,173,2095,...,6019629,1533645.0,30.0,366931.0,8569.0,4466174.0,1553455.0,ITH,ITH3,4.907704e+06
4,2021-03-26T17:00:00,6,Friuli Venezia Giulia,650,78,728,15292,16020,52,768,...,1583938,566752.0,5.0,83507.0,11473.0,1410324.0,173614.0,ITH,ITH4,1.211357e+06
5,2021-03-26T17:00:00,7,Liguria,605,67,672,5654,6326,171,487,...,1204780,505596.0,8.0,87194.0,0.0,1034593.0,170187.0,ITC,ITC3,1.543127e+06
6,2021-03-26T17:00:00,8,Emilia-Romagna,3515,399,3914,67945,71859,24,2391,...,4721424,1660353.0,26.0,326563.0,213.0,3919926.0,801498.0,ITH,ITH5,4.467118e+06
7,2021-03-26T17:00:00,9,Toscana,1473,257,1730,25496,27226,362,1425,...,3312214,1725391.0,14.0,187035.0,1765.0,2841708.0,470506.0,ITI,ITI1,3.722729e+06
8,2021-03-26T17:00:00,10,Umbria,376,57,433,4817,5250,83,320,...,957768,334692.0,2.0,50127.0,0.0,773860.0,183908.0,ITI,ITI2,8.802850e+05
9,2021-03-26T17:00:00,11,Marche,807,152,959,9153,10112,-64,674,...,982389,596994.0,6.0,85770.0,0.0,887712.0,94677.0,ITI,ITI3,1.518400e+06


In [12]:
# Update icu csv data
df_icu_updated_r = pd.DataFrame.from_dict(F.icu_data())
df_icu_updated_r.to_csv(icu_regions_name, index=False)

In [13]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, left_on='denominazione_regione', right_on='regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,tamponi_test_antigenico_rapido,codice_nuts_1,codice_nuts_2,popolazione,regione,ricoverati_area_non_critica,posti_letto_area_non_critica,ricoverati_terapia_intensiva,posti_letto_terapia_intensiva,posti_letto_terapia_intensiva_attivabili
0,2021-03-26T17:00:00,1,Piemonte,3759,358,4117,31018,35135,-157,2117,...,860533.0,ITC,ITC1,4.341375e+06,Piemonte,3759,5824,358,628,99
1,2021-03-26T17:00:00,2,Valle d'Aosta,24,7,31,659,690,41,57,...,11487.0,ITC,ITC2,1.255010e+05,Valle d'Aosta,24,227,7,30,6
2,2021-03-26T17:00:00,3,Lombardia,7111,848,7959,96696,104655,2577,5077,...,802360.0,ITC,ITC4,1.010397e+07,Lombardia,7111,13610,848,1416,114
3,2021-03-26T17:00:00,5,Veneto,1548,257,1805,37551,39356,173,2095,...,1553455.0,ITH,ITH3,4.907704e+06,Veneto,1548,6000,257,1000,0
4,2021-03-26T17:00:00,6,Friuli Venezia Giulia,650,78,728,15292,16020,52,768,...,173614.0,ITH,ITH4,1.211357e+06,Friuli Venezia Giulia,650,1277,78,175,0
5,2021-03-26T17:00:00,7,Liguria,605,67,672,5654,6326,171,487,...,170187.0,ITC,ITC3,1.543127e+06,Liguria,605,1785,67,222,12
6,2021-03-26T17:00:00,8,Emilia-Romagna,3515,399,3914,67945,71859,24,2391,...,801498.0,ITH,ITH5,4.467118e+06,Emilia-Romagna,3515,6515,399,760,0
7,2021-03-26T17:00:00,9,Toscana,1473,257,1730,25496,27226,362,1425,...,470506.0,ITI,ITI1,3.722729e+06,Toscana,1473,5033,257,618,17
8,2021-03-26T17:00:00,10,Umbria,376,57,433,4817,5250,83,320,...,183908.0,ITI,ITI2,8.802850e+05,Umbria,376,860,57,139,2
9,2021-03-26T17:00:00,11,Marche,807,152,959,9153,10112,-64,674,...,94677.0,ITI,ITI3,1.518400e+06,Marche,807,1360,152,250,16


In [14]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [15]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['posti_letto_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['posti_letto_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2021-03-26T17:00:00,3,Lombardia,7111,848,7959,96696,104655,2577,5077,...,59.89,2595,31,60804,13912.0,100,2400,8.35,0.05025,7.11
1,2021-03-26T17:00:00,8,Emilia-Romagna,3515,399,3914,67945,71859,24,2391,...,52.50,69,321,32068,4737.0,46,2312,7.46,0.05352,7.32
2,2021-03-26T17:00:00,16,Puglia,1766,231,1997,42799,44796,619,2162,...,40.60,615,129,12135,5480.0,41,1502,17.82,0.05394,4.61
3,2021-03-26T17:00:00,1,Piemonte,3759,358,4117,31018,35135,-157,2117,...,57.01,-240,-465,32115,8008.0,38,2236,6.59,0.04876,6.90
4,2021-03-26T17:00:00,5,Veneto,1548,257,1805,37551,39356,173,2095,...,25.70,180,234,42645,6814.0,33,1889,4.91,0.04269,7.65
5,2021-03-26T17:00:00,12,Lazio,2792,360,3152,46067,49219,158,2006,...,38.18,124,-49,35618,21426.0,37,1811,5.63,0.03420,4.72
6,2021-03-26T17:00:00,15,Campania,1586,181,1767,94484,96251,-989,1947,...,29.19,-996,-121,21993,13214.0,42,2894,8.85,0.03365,5.67
7,2021-03-26T17:00:00,9,Toscana,1473,257,1730,25496,27226,362,1425,...,41.59,359,-195,26050,10265.0,22,1041,5.47,0.03828,5.07
8,2021-03-26T17:00:00,19,Sicilia,799,121,920,15483,16403,409,892,...,14.51,420,-3,27829,6658.0,22,461,3.21,0.01795,3.42
9,2021-03-26T17:00:00,6,Friuli Venezia Giulia,650,78,728,15292,16020,52,768,...,44.57,47,-74,10449,3430.0,28,688,7.35,0.06340,7.84


In [16]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [17]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'posti_letto_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,posti_letto_terapia_intensiva,saturazione_terapia_intensiva
0,Marche,152,98,250,60.80
1,Lombardia,848,568,1416,59.89
2,Piemonte,358,270,628,57.01
3,Emilia-Romagna,399,361,760,52.50
4,P.A. Trento,44,46,90,48.89
5,Friuli Venezia Giulia,78,97,175,44.57
6,Molise,17,22,39,43.59
7,Toscana,257,361,618,41.59
8,Umbria,57,82,139,41.01
9,Puglia,231,338,569,40.60


In [18]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['variazione_totale_positivi'] / (df['totale_positivi'] - df['variazione_totale_positivi'])).replace(np.inf, 0) * 100).round(2)
    return df.fillna(0)

In [19]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../../data/csv/regioni/covid19-veneto.csv
Aggiornato ../../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../../data/csv/regioni/covid19-liguria.csv
Aggiornato ../../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../../data/csv/regioni/covid19-toscana.csv
Aggiornato ../../data/csv/regioni/covid19-umbria.csv
Aggiornato ../../data/csv/regioni/covid19-marche.csv
Aggiornato ../../data/csv/regioni/covid19-lazio.csv
Aggiornato ../../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../../data/csv/regioni/covid19-molise.csv
Aggiornato ../../data/csv/regioni/covid19-campania.csv
Aggiornato ../../data/csv/regioni/covid19-puglia.csv
Aggiornato ../../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../../data/csv/regioni/covid19-calabria.csv
Aggiornato ../../data/csv/regioni/covid19-sicilia.csv


In [20]:
# df_tb = df_rd[df_rd.codice_regione > 20].reset_index(drop=True)
# df_tb

In [21]:
# Initialize df_taa
# df_taa = Functions.replace_bt_with_taa(df_tb[:2])
# for i in range(2, df_tb.shape[0], 2):
#     df_taa = df_taa.append(Functions.replace_bt_with_taa(df_tb[i:i+2]), ignore_index=True)
# df_taa = add_cols(df_taa)
# df_taa

In [22]:
# region_name = df_taa.denominazione_regione.iloc[0].lower()
# csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
# df_taa.to_csv(csv_out_region_name, index=False)
# print('Aggiornato {file}'.format(file=csv_out_region_name))